In [1]:
import time
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import tushare as ts
ts.set_token('29eaf3bcac23df4c6d025de157ab2d53beead3391fbe6e83b4ebcb6c')
pro = ts.pro_api()

import mpl_finance as mpf
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.pylab import date2num
import matplotlib as mpl
import seaborn as sns

D:\Users\admin\Anaconda3\envs\py37-stock\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


In [9]:
# read 
from mylab.stock.myread import getIndexBasic # remote or local
from mylab.stock.myread import getIndexDaily
from mylab.stock.myread import getIndexWeekly
from mylab.stock.myread import getIndexMonthly
from mylab.stock.myread import getIndex
from mylab.stock.myread import getIndustryBasic  #local only
from mylab.stock.myread import getIndustryDaily
from mylab.stock.myread import getIndustryWeekly
from mylab.stock.myread import getIndustryMonthly
from mylab.stock.myread import getIndustry
from mylab.stock.myread import getStock
# plot 
from mylab.stock.myplot import plotStock
from mylab.stock.myplot import displaySelect
# feature
from mylab.stock.myfeature import getMacd
from mylab.stock.myfeature import getMa
from mylab.stock.myfeature import getKdj
from mylab.stock.myfeature import extractFeature

In [ ]:
interest_df = pd.read_csv("./data/stock/blue_stock.csv")
inteest_df.head()

In [4]:
def selectStockByKdj(df, stock_monthly_KDJ = True, stock_weekly_KDJ = True ):
    select_df = df.copy(deep = True)
    select_df = select_df.loc[select_df["kdj_signal"] == 1, :]
    select_df = select_df.loc[select_df["D_monthly"].values < 60, :]
    select_df = select_df.loc[select_df["D_weekly"].values < 60, :]
    if stock_monthly_KDJ:
        select_df = select_df.loc[select_df["K_monthly"].values >= select_df['D_monthly'].values, :]
    else:
        select_df = select_df.loc[select_df["K_monthly"].values < select_df['D_monthly'].values, :]
    if stock_weekly_KDJ:
        select_df = select_df.loc[select_df["K_weekly"].values >= select_df['D_weekly'].values, :]
    else:
        select_df = select_df.loc[select_df["K_weekly"].values < select_df['D_weekly'].values, :]
    
    return select_df

In [16]:
start_date = "20140101"
end_date = "20200324"

stock_select_df = pd.read_csv("./data/stock/blue_stock.csv")
all_select_df = pd.DataFrame()
for i in range(len(stock_select_df)):
    stock_code = stock_select_df.loc[i,"ts_code"]
    stock_name = stock_select_df.loc[i,"name"]

    stock_df = getStock(stock_code,start_date, end_date, LOCAL = True)
#     stock_df = selectStockByKdj(stock_df, stock_monthly_KDJ = True, stock_weekly_KDJ = True )  # stock 
    stock_df = stock_df.loc[stock_df["kdj_signal"] == 1, :]
    stock_df = stock_df.loc[stock_df["D"].values < 20, :]
    print(stock_code,len(stock_df))
    if stock_code[0] == "3":
        continue
    if stock_name[0] == "*":
        continue     
    if len(stock_df) > 0:  
        all_select_df  = pd.concat([all_select_df,stock_df],axis = 0)

all_select_df.to_csv("./output/stock_only/"+"blue_stock_only.csv", index = False)
print(all_select_df.shape)
all_select_df = all_select_df.loc[all_select_df["trade_date"] == end_date,:]
print(all_select_df.shape)
all_select_df.head()

002050.SZ 13
002236.SZ 19
002262.SZ 13
002415.SZ 16
600183.SH 13
601888.SH 6
603866.SH 8
300596.SZ 7
603517.SH 4
603816.SH 15
002878.SZ 9
603833.SH 9
(125, 60)
(0, 60)


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,...,K_monthly,D_monthly,J_monthly,kdj_signal_monthly,sema_monthly,lema_monthly,dif_monthly,dea_monthly,macd_monthly,macd_signal_monthly


In [17]:
save_dir = "./output/stock_only/"
start_date = "20140101"
# end_date = "20200324"

# all_select_df = pd.read_csv("./output/stock_only/"+"blue_stock_only.csv")
interest_df = pd.read_csv("./data/stock/blue_stock.csv")
stocks_code = interest_df.ts_code.values
# stocks_code = ["603969.SH"]
for stock_code in stocks_code:
    stock_df = getStock(stock_code,start_date, end_date , LOCAL = True)
    select_df = pd.read_csv("./output/stock_only/"+"blue_stock_only.csv")
    select_df["trade_date"] = select_df["trade_date"].astype("str")
    select_df = select_df.loc[select_df["ts_code"].values == stock_code,["trade_date","pre_close","pct_chg"]]
    select_df = pd.merge(stock_df,select_df, how = "left", on = "trade_date")
    select_df = select_df.dropna(how = "any")
    print(stock_code,len(select_df))
    for i in range(len(select_df)):
        ix = select_df.index[i]
        trade_date = select_df.trade_date.values[i]
        display_df = displaySelect(stock_df,ix, trade_date= trade_date, save_dir = save_dir )
        plt.close()                

002050.SZ 12
002236.SZ 18
002262.SZ 13
002415.SZ 16
600183.SH 13
601888.SH 6
603866.SH 7
300596.SZ 0
603517.SH 3
603816.SH 15
002878.SZ 8
603833.SH 9
